In [4]:
import numpy as np
import pandas as pd
import seaborn  as sns
import matplotlib.pyplot as plt
np.random.seed(sum(map(ord, "aesthetics")))
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.preprocessing import scale
import scipy as sc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import scale
import os


def parse_response(str_id):
    if str_id.split('_')[1] == 'resp':
        return 1
    return 0
   
    
def preprocess_dataset(dataset):
    gen_names = dataset["SYMBOL"].as_matrix()
    responses = np.zeros(dataset.shape[1]-1, dtype=np.int)
    target_ids = dataset.columns[1:].values
    feature_matrix = np.zeros((dataset.shape[1]-1, dataset.shape[0]), dtype=np.float)
    for idx, target_id in enumerate(target_ids):
        feature_matrix[idx, :] = dataset[target_id].values
        responses[idx] = parse_response(target_id)
    return feature_matrix, responses, target_ids, gen_names


def get_gen_hists(xpression, cl_masks, bins=10):
    xpr_range = (xpression.min(), xpression.max())
    hists = []
    for mask in cl_masks:
        hist, _ = np.histogram(xpression[mask], bins=bins, range=xpr_range)
        hists.append(hist)
    return hists


def get_pdf_sim(p, q):
    return -q.dot(np.log2(p)) - p.dot(np.log2(q))
    # return sc.stats.entropy(p, q)

    
def xpressions_to_order_matrix(xpressions):
    orders_num = np.arange(xpressions.shape[1], dtype=np.int32)
    order_matrix = np.zeros_like(xpressions, dtype=np.int32)
    for row in range(xpressions.shape[0]):
        ordered_ids = np.array([ind 
                          for ind, xpr in sorted([(ind, xpr) 
                                                   for ind, xpr in enumerate(xpressions[row,:])],
                                                 key=lambda p: p[1])])
        order = np.zeros_like(ordered_ids, dtype=np.int32)
        order[ordered_ids] = np.arange(xpressions.shape[1])
        order_matrix[row, :] = order
    return order_matrix
    
    
def get_relation_matrix(order_seq):
    matrix = np.zeros(shape=(order_seq.shape[0], order_seq.shape[0]), dtype=np.int32)    
    for row in range(matrix.shape[0]):
        for col in range(matrix.shape[1]):
            matrix[row, col] = order_seq[row] - order[col]
    return matrix


def get_relation_cumul_matrix(order_matrix):
    matrix = np.zeros(shape=(order_matrix.shape[1], order_matrix.shape[1]), dtype=np.float32)    
    for row in range(matrix.shape[1]):        
        for col in range(matrix.shape[1]):
            matrix[row, col] = (order_matrix[:, row] - order_matrix[:, col]).mean()
        if row % 1000 == 0:
            print(row)
    return matrix        
# np.histogram(np.array([0,0,0,1,1,2,1]),bins=2, range=(0,1))


def get_pair_value(order_matrix, pair, responses, pr=None):
    if not pr:
        pr = (responses == 1).mean()
        
    p_ab = np.mean(order_matrix[:, pair[0]] > order_matrix[:, pair[1]])
    p_ab_r = np.mean(order_matrix[responses==1, pair[0]] > order_matrix[responses==1, pair[1]])
    pr_ab = (p_ab_r * pr) / p_ab
    
    p_ba = np.mean(order_matrix[:, pair[0]] < order_matrix[:, pair[1]])
    p_ba_r = np.mean(order_matrix[responses==1, pair[0]] < order_matrix[responses==1, pair[1]])
    pr_ba = (p_ba_r * pr) / p_ba
    return pr_ab, pr_ba


def xpression_to_order_feature(xpressions, pairs, gens):
    order_features = np.zeros(shape=(xpressions.shape[0], len(pairs)))
    features_name = []
    for idx, pair in enumerate(pairs):
        order_features[:, idx] = xpressions[:, pair[0]] > xpressions[:, pair[1]] + 0
        features_name.append("compare_expr({}, {})".format(gens[pair[0]], gens[pair[1]]))
    return order_features, features_name
                                            
    

def get_metrics(conf):
    """return tuple (precision 0 class, recall 0 class, precision 1 class, recall 1 class)"""
    prec_0 = conf[0,0]/(conf[1,0] + conf[0,0])
    if (conf[1,0] + conf[0,0]) == 0:
        prec_0 = 0
    recall_0 = conf[0,0]/(conf[0,0] + conf[0,1])
    prec_1 = conf[1,1]/(conf[1,1] + conf[0,1])
    recall_1 = conf[1,1]/(conf[1,1] + conf[1,0])
    if (conf[1,1] + conf[0,1]) == 0:
        prec_1 = 0
    accuracy = (conf[0,0] + conf[1, 1]) / (conf.sum())
    # print(prec_0, recall_0, prec_1, recall_1)
    return prec_0, recall_0, prec_1, recall_1


def get_splits(responses, k=10):
    nonresp = np.arange(responses.shape[0])[responses==0]
    resp = np.arange(responses.shape[0])[responses==1]
    np.random.shuffle(nonresp)
    np.random.shuffle(resp)
    folds = []
    for i in range(9):
        folds.append(np.hstack((resp[i*22:i*22+22],nonresp[i*5:i*5+5])))
    folds.append(np.hstack((resp[9*22:],nonresp[9*5:])))
    cases = []
    for i in range(10):
        test = folds[i]
        train = np.hstack((fold for j, fold in enumerate(folds) if i != j))
        cases.append( (train, test) )                      
    return folds, cases


def do_simmetry_class_cv(X, y, clf, iters=10, cv=10, feature_maker=lambda  X, y, train, 
                         test: (X, "")):
    nonresp = np.arange(y.shape[0])[y==0]
    resp = np.arange(y.shape[0])[y==1]
    results_a = []
    results_b = []
    cv_results = []
    accs = []
    # print(nonresp)
    all_descriptions = []
    for it in range(iters):
        print("iteration: {}".format(it))
        np.random.shuffle(resp)
        np.random.shuffle(nonresp)
        indicies = np.hstack((nonresp, resp[:nonresp.shape[0]]))
        # print(indicies)
        stat, common_stat, cv_result, descriptions = do_cross_val(X[indicies, :], 
                     y[indicies],
                     clf, cv, feature_maker)
        results_a.append(stat.mean(axis=0))        
        # print(stat.mean(axis=0))
        results_b.append(np.array(common_stat))    
        cv_results.append(cv_result)        
        all_descriptions.append(descriptions)
    return np.vstack(results_a), np.vstack(results_b), cv_results, all_descriptions


def process_results(a, b, cv_results):
    a_aucs = []    
    a_accs = []
    b_aucs = []    
    b_accs = []
    for result in cv_results:
        actuals = []
        preds = []
        probs = []
        for actual, predicted, proba in result:
            a_accs.append((actual == predicted).mean())
            actuals.append(actual)
            preds.append(predicted)
            probs.append(proba)
            fpr, tpr, _ = roc_curve(actual, proba)
            a_aucs.append(auc(fpr, tpr))         
        b_accs.append((np.hstack(actuals)==np.hstack(preds)).mean())    
        fpr, tpr, thresh = roc_curve(np.hstack(actuals), np.hstack(probs))
        b_aucs.append(auc(fpr, tpr))
    columns=["precision 0", "recall 0", "precision 1", "recall 1", "accuracy", "auc"]
    rows = ["схема а", "схема б"]
    data = np.zeros((len(rows), len(columns)))
    data[0, :4] = a.mean(axis=0)
    data[0, 4:] = [np.mean(a_accs), np.mean(a_aucs)]
    data[1, :4] = b.mean(axis=0)
    data[1, 4:] = [np.mean(b_accs), np.mean(b_aucs)]    
    table = pd.DataFrame(data, columns=columns, index=rows)
    
    return table


def do_cross_val(X, y, clf,  cv=10, feature_maker=lambda X, y, test, train: (X, "")):
    skfold = StratifiedKFold(n_splits=cv, shuffle=True)    
    cv_results = []
    # print ((y==1).sum(), (y==0).sum())
    skfold.get_n_splits(X, y)
    descriptions = []
    for train, test in skfold.split(X, y):        
        features, description = feature_maker(X, y, train, test)
        descriptions.append(description)
        clf.fit(features[train, :], y[train])
        prediction = clf.predict(features[test, :])
        proba = clf.predict_proba(features[test, :])
        cv_results.append((y[test], prediction, proba[:, 1]))
        # print(y[test].shape, (y[test] == 0).sum())        
    conf_m =  confusion_matrix(cv_results[0][0], cv_results[1][0]) * 0
    stat = []    
    fpr_tpr = []
    tpr = []
    accs = []
    for actual, predicted, proba in cv_results:
        conf =  confusion_matrix(actual, predicted)
        stat.append(np.array(get_metrics(conf)))
        conf_m = conf_m + conf        
    # print((conf_m[0, 0] + conf_m[1, 1])/conf_m.sum())    
    return np.vstack(stat), get_metrics(conf_m), cv_results, descriptions


def get_splits(responses, k=10):
    nonresp = np.arange(responses.shape[0])[responses==0]
    resp = np.arange(responses.shape[0])[responses==1]
    np.random.shuffle(nonresp)
    np.random.shuffle(resp)
    folds = []
    for i in range(9):
        folds.append(np.hstack((resp[i*22:i*22+22],nonresp[i*5:i*5+5])))
    folds.append(np.hstack((resp[9*22:],nonresp[9*5:])))
    cases = []
    for i in range(10):
        test = folds[i]
        train = np.hstack((fold for j, fold in enumerate(folds) if i != j))
        cases.append( (train, test) )                      
    return folds, cases
    
    
def get_n_biggest_sims_ids(sims, subset, n=140):
    subsims = sims[subset]
    bound = np.sort(subsims)[-n]    
    return subset[subsims >= bound][:n]


def read_GEO_dataset(pathto_xpr, pathto_resp, pathto_nonresp, prefix=""):
    geo_dataset = pd.read_csv(os.path.join(prefix, pathto_xpr), sep=" ")    
    with open(os.path.join(prefix, pathto_resp), "r") as respf,\
         open(os.path.join(prefix, pathto_nonresp), "r") as nonrespf:
        resp = set(respf.read().split())
        nonresp = set(nonrespf.read().split())
    responses = np.array(list(map(lambda x: 1 if x in resp else int(x in nonresp) - 1 ,
                                  geo_dataset.columns[1:])))
    feature_matrix = geo_dataset[geo_dataset.columns[1:]].get_values().T
    target_ids = geo_dataset.columns[1:].values
    gen_names = geo_dataset["Group.1"].as_matrix()
    return feature_matrix, responses, target_ids, gen_names


# def test_features(X, y, features_builder, clf=None):
def heuristics_1_2_features(X, y, train, test):
    description = "экспрессии генов отобранных по пересечению 1-й и второй эвристик\n id генов:\n"
    gen_res = []
    for g in range(X.shape[1]):
        lg = LogisticRegression(penalty='l1', C=1)
        lg.fit(X[train, g:g+1], y[train])
        gen_res.append((lg.predict(X[train, g:g+1]) == y[train]).mean())
    gen_res = np.array(gen_res)
    good_h1 = np.arange(X.shape[1])[gen_res > y[train].mean()]
    
    gen_corr = []
    for g in range(X.shape[1]):    
        gen_corr.append(abs(np.corrcoef(X[train, g], y[train])[0,1]))
    gen_corr = np.array(gen_corr)  
    good_h2 = np.arange(X.shape[1])[gen_corr > np.percentile(gen_corr, 97.5)]
    good = np.array([g for g in set(good_h1).intersection(set(good_h2))])   
    # print(good)
    description = description + "\t".join([str(gid) for gid in good])
    return X[:, good], description
    
   
def heuristics_4_features(X, y, train, test, topn=70):
    print("heuristic 4 started")
    order_matrix = xpressions_to_order_matrix(X[train,:])
    value_matrix = np.zeros(shape=(order_matrix.shape[1], order_matrix.shape[1]),
                            dtype=np.float32)
    for i in range(order_matrix.shape[1]):
        for j in range(i+1, order_matrix.shape[1]):
            pr_ab, pr_ba = get_pair_value(order_matrix, (i,j), y[train])
            value_matrix[i, j] = pr_ab
            value_matrix[j, i] = pr_ba
    if i % 5000 == 0:
        print("\tcomplete: {} rows".format(str(i)))        
    
    new_val = value_matrix + 0.
    for i in range(order_matrix.shape[1]):
        for j in range(order_matrix.shape[1]):
            if np.isnan(value_matrix[i,j]):
                new_val[i, j] = 0.
                new_val[j, i] = 0.
                
    dif_matrix = new_val - new_val.T
    p = np.percentile(dif_matrix, 99.99)
    pairs_val = []
    for i in range(order_matrix.shape[1]):
        for j in range(order_matrix.shape[1]):
            if dif_matrix[i, j] >= p:       
                pairs_val.append(((i, j), dif_matrix[i, j]))
    
    
    pairs_val = sorted(pairs_val, key=lambda p_v: -p_v[1])                    
    good_pairs = [p_v[0] for p_v in pairs_val[:topn]]
    order_features, pair_names = xpression_to_order_feature(X, good_pairs, np.arange(X.shape[1]))
    description = "\t".join(pair_names)
    return order_features, description


def heuristics_4_features_rnd(X, y, train, test, topn=70):
    print("heuristic 4 started")
    good_pairs = []
    choosen = np.random.choice(np.arange(X.shape[1]), size=2*topn, replace=False)
    for i in range(0, 2*topn,2):
        good_pairs.append((choosen[i], choosen[i+1]))
    order_features, pair_names = xpression_to_order_feature(X, good_pairs, np.arange(X.shape[1]))
    description = "\t".join(pair_names)
    return order_features, description



In [2]:
# columns = ["precision 0", "recall 0", "precision 1", "recall 1", "accuracy", "auc"]
# pd.DataFrame(np.array([0,1,2,3,4,5]).reshape((-1,6)), columns=columns, index=["a"])
# np.hstack([cv_results[0][0][0], cv_results[0][0][1]])
# prefix = "."#dataset/breast_cancer_paclitaxel_plus_radiation/
# pathto_xpr = "dataset/breast_cancer_paclitaxel_plus_radiation/GSE22513_expression_by_symbol.txt"
# pathto_resp = "dataset/breast_cancer_paclitaxel_plus_radiation/responders.txt"
# pathto_nonresp = "dataset/breast_cancer_paclitaxel_plus_radiation/nonresponders.txt"
# xpressions, responses,target_ids, gens = read_GEO_dataset(pathto_xpr, 
#                                                           pathto_resp,
#                                                           pathto_nonresp,
#                                                           prefix)
# mask = range(0,responses.shape[0],2)
# xpressions, responses,target_ids = xpressions[mask, :], responses[mask], target_ids[mask]
# np.random.choice()

In [5]:
path_to_dataset = "dataset/GSE20194_breastCancer_TFAC.txt"
dataset = pd.read_csv(path_to_dataset, sep='\t')
xpressions, responses, target_ids, gens = preprocess_dataset(dataset)
perm = np.arange(xpressions.shape[0])
np.random.shuffle(perm)
xpressions = xpressions[perm, :]
responses = responses[perm]
target_ids = target_ids[perm]

In [4]:
"""
GSM559042	non-pCR breast biopsy, sample 1 rep 1
GSM559043	non-pCR breast biopsy, sample 1 rep 2
GSM559044	non-pCR breast biopsy, sample 2 rep 1
GSM559045	non-pCR breast biopsy, sample 2 rep 2
GSM559046	non-pCR breast biopsy, sample 3 rep 1
GSM559047	non-pCR breast biopsy, sample 3 rep 2
GSM559048	non-pCR breast biopsy, sample 4 rep 1
GSM559049	non-pCR breast biopsy, sample 4 rep 2
GSM559050	non-pCR breast biopsy, sample 5 rep 1
GSM559051	non-pCR breast biopsy, sample 5 rep 2
GSM559052	non-pCR breast biopsy, sample 6 rep 1
GSM559053	non-pCR breast biopsy, sample 6 rep 2
GSM559054	non-pCR breast biopsy, sample 8 rep 1
GSM559055	non-pCR breast biopsy, sample 8 rep 2
GSM559056	non-pCR breast biopsy, sample 9 rep 1
GSM559057	non-pCR breast biopsy, sample 9 rep 2
GSM559058	non-pCR breast biopsy, sample 10 rep 1
GSM559059	non-pCR breast biopsy, sample 10 rep 2
GSM559060	non-pCR breast biopsy, sample 13 rep 1
GSM559061	non-pCR breast biopsy, sample 13 rep 2
GSM559062	pCR breast biopsy, sample 15 rep 1
GSM559063	pCR breast biopsy, sample 15 rep 2
GSM559064	pCR breast biopsy, sample 16 rep 1
GSM559065	pCR breast biopsy, sample 16 rep 2
GSM559066	pCR breast biopsy, sample 18 rep 1
GSM559067	pCR breast biopsy, sample 18 rep 2
GSM559068	pCR breast biopsy, sample 19 rep 1
GSM559069	pCR breast biopsy, sample 19 rep 2
"""
target_ids

array(['Tumour_resp_GSM505377', 'Tumour_resp_GSM505383',
       'Tumour_resp_GSM505577', 'Tumour_resp_GSM505389',
       'Tumour_resp_GSM505394', 'Tumour_resp_GSM505410',
       'Tumour_nonresp_GSM505566', 'Tumour_resp_GSM505432',
       'Tumour_nonresp_GSM505387', 'Tumour_resp_GSM505465',
       'Tumour_resp_GSM505544', 'Tumour_resp_GSM505529',
       'Tumour_resp_GSM505353', 'Tumour_resp_GSM505515',
       'Tumour_nonresp_GSM505496', 'Tumour_nonresp_GSM505447',
       'Tumour_nonresp_GSM505358', 'Tumour_nonresp_GSM505360',
       'Tumour_resp_GSM505479', 'Tumour_nonresp_GSM505344',
       'Tumour_resp_GSM505402', 'Tumour_resp_GSM505506',
       'Tumour_resp_GSM505474', 'Tumour_resp_GSM505599',
       'Tumour_resp_GSM505572', 'Tumour_resp_GSM505408',
       'Tumour_resp_GSM505605', 'Tumour_resp_GSM505404',
       'Tumour_nonresp_GSM505478', 'Tumour_resp_GSM505340',
       'Tumour_nonresp_GSM505449', 'Tumour_nonresp_GSM505351',
       'Tumour_resp_GSM505593', 'Tumour_nonresp_GSM505459'

In [6]:
x, d = heuristics_1_2_features(xpressions, responses, np.arange(xpressions.shape[0]), [])
print(d)

экспрессии генов отобранных по пересечению 1-й и второй эвристик
 id генов:
8711	1544	12816	7186	9746	8729	11291	1052	3623	11816	4649	8745	8235	8236	10793	561	4146	4148	6719	7743	8257	8778	4683	3146	1613	5200	2130	12886	10327	1112	5728	11872	12898	10343	9836	5746	6269	2689	7813	6791	4745	8841	9865	11918	6799	3218	9880	12441	1181	10405	167	10412	4269	10926	3245	1708	7854	2227	2228	3263	9919	5825	3266	11458	12998	13517	2766	1743	12497	7897	5340	1249	9955	3302	3303	13550	6895	1779	1273	7932	13056	13057	9480	4874	7946	9486	2322	5907	6430	4384	7458	8483	2854	11563	4396	813	8500	7485	4928	10056	6473	10058	2379	3918	10068	9557	3926	7002	13150	7007	12127	1889	9570	13160	1896	11114	10604	13168	4466	13170	6005	6008	6009	12666	12667	3965	2943	9600	9602	11651	7044	6018	7560	12696	13213	7073	7075	1447	941	4014	2991	1969	5555	4020	3510	6074	3515	6080	7105	3523	7114	1997	4560	2000	10199	13283	10726	8166	7656	11753	2026	3055	5616	11759	6642	12793	9725	2559


In [98]:
# Эвристика 1:
gen_res = []
for g in range(xpressions.shape[1]):
    lg = LogisticRegression(penalty='l1', C=1)
    lg.fit(xpressions[:, g:g+1], responses)
    gen_res.append((lg.predict(xpressions[:, g:g+1]) == responses).mean())

In [99]:
gen_res = np.array(gen_res)
good_h1 = np.arange(gens.shape[0])[gen_res > responses.mean()]
good_h1.shape

(710,)

In [107]:
# Эвристика 2
gen_corr = []
for g in range(xpressions.shape[1]):    
    gen_corr.append(abs(np.corrcoef(xpressions[:, g], responses)[0,1]))
gen_corr = np.array(gen_corr)  
good_h2 = np.arange(gens.shape[0])[gen_corr > np.percentile(gen_corr, 95)]
good_h2.shape

(692,)

In [109]:
# пересечение эвристик 1 2
interscetion_h1_h2 = np.array([id for id in set(good_h1).intersection(set(good_h2))], dtype=np.int)
interscetion_h1_h2.shape

(280,)

In [12]:
# Эвристика 3
resp_mask = responses == 1
nonresp_mask = responses == 0

gen_pdfs = []
nb_bins = 15
for g in range(xpressions.shape[1]):    
    hists = get_gen_hists(xpressions[:, g], [resp_mask, nonresp_mask], bins=nb_bins)    
    p = (hists[0] + 1) / (hists[0].sum() + hists[0].shape[0]) 
    q = (hists[1] + 1) / (hists[1].sum() + hists[1].shape[0])
    gen_pdfs.append((p, q))
#     gen_pdf_sims.append(get_pdf_sim(p, q))
    if g % 1000 == 0:
         print(g)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000


In [14]:
gen_pdf_sims = []
for gen_pdf in gen_pdfs:
    gen_pdf_sims.append(get_pdf_sim(gen_pdf[0], gen_pdf[1]))        
gen_pdf_sims = np.array(gen_pdf_sims)

In [15]:
good_h3 = get_n_biggest_sims_ids(gen_pdf_sims, np.arange(gens.shape[0]), gen_pdf_sims.shape[0]// 100)
good_h3.shape

(209,)

Рассматривается датасет GSE20194_breastCancer_TFAC, в нем данные по химиотерапии рака молочной железы. В данных два класса: ответчики и неответчики, размер классов 222 и 56 соответственно. В качестве признаков  есть информация по 13832 генам

Вторая итерация:
** мысли:
1. посмотреть на порядок генов
2. надо подумать как можно посчитать похожесть генов, и про перевод в некоторое векторное пространство
3. если удалось перевести какое-то векторное пространство то туже можно подумать про ядра на последовательностях 
   и разные сеточки **
   
##### 1. порядок генов:

После собрания стало известно что использовалась хитрая нормализация (квантильная???), такая что по сути сохранился только порядок экспрессий генов, а сами значения по сути не особо важны. 
Тут думаю стоит посмотреть нет ли какой-либо взаимосвязи между взаимным расположением генов в ряду генов осортированных по уровню экспрессии и ответом на терапию.

Пока в голову пришла только мысль сделать аналог матрицы смежности для генов по группам, где на позиции (i, j) будет стоять усредненная по группе разность в порядковых номере i-го и j-го генов в упорядоченном по возрастанию экспрессии ряду генов.

Усреднение по группам по тому что если делать для каждого отдельно наверно надо оочень много памяти

*Да еще комментарий: если смотреть просто порядковый номер гена, то это видимо будет тоже самое что было с простым использованием экспрессий...*

Далее думаю имеет смысл посмотреть есть есть ли какие-то специфичные группе пары генов по расположению, а еще лучше какие-либо n-граммы.

- посмотрел на пары у которых большая разница в порядковых номерах для ответчиков и неответчиков
- далее решил сделать для генов вывод pr(response==1 | A > B), pr(response==1 | A < B), где A = order_num(gen_a) ... - то есть порядковый номер гена A в последвательности, для B аналогично. Вроде есть интересные моменты например для генов 2 и 1, получилось что вероятность ответа 1 при условии что order_num(ген1) < order_num(ген2)
- а может сразу сроить выводы pr(response==1 | A > B) и pr(response==1 | A < B) вместо матрицы смежности?


In [16]:
order_matrix = xpressions_to_order_matrix(xpressions)
order_matrix.shape

(14, 20993)

In [17]:
value_matrix = np.zeros(shape=(order_matrix.shape[1], order_matrix.shape[1]),  dtype=np.float32)

In [18]:
for i in range(order_matrix.shape[1]):
    for j in range(i+1, order_matrix.shape[1]):
        pr_ab, pr_ba = get_pair_value(order_matrix, (i,j), responses)
        value_matrix[i, j] = pr_ab
        value_matrix[j, i] = pr_ba
    if i % 1000 == 0:
        print("complete: {} rows".format(str(i)))

complete: 0 rows


KeyboardInterrupt: 

In [63]:
good_pairs = []
a=0.22
eps = 0.003
b = a + eps
for i in range(order_matrix.shape[1]):
    for j in range(i+1, order_matrix.shape[1]):
        if value_matrix[i,j] == np.nan:
            print(i,j)
            break
           
        if (value_matrix[i,j]) > a and (value_matrix[i,j] < b):                
            good_pairs.append((i, j))
        
len(good_pairs)

942

In [119]:
mask = responses == 1
sum(list(order_matrix[mask, 14] < order_matrix[mask, 12549]))
value_matrix[14, 12549], value_matrix[12549, 14]

(0.22222222, 0.81784385)

In [67]:
val_without_na = value_matrix + 0.0
val_without_na[np.isnan(value_matrix)] = 0.7986
# dif_matrix[np.isnan(dif_matrix)] = 0
dif_matrix = val_without_na - val_without_na.T

In [93]:
np.save("dif_matrix", dif_matrix)

In [88]:
# np.isnan(dif_matrix).sum()
# np.isnan(val_without_na).sum()
# 0.820, 0.822, 0.826
good_pairs = []
for i in range(order_matrix.shape[1]):
    for j in range(order_matrix.shape[1]):
        if dif_matrix[i, j] > 0.822:                
            good_pairs.append((i, j))

In [286]:
order_features3, pair_names = xpression_to_order_feature(xpressions, good_pairs3, gens)
order_features2, pair_names = xpression_to_order_feature(xpressions, good_pairs2, gens)
order_features1, pair_names = xpression_to_order_feature(xpressions, good_pairs, gens)

In [133]:
# symmetr_res_a.mean(axis=0), symmetr_res_b.mean(axis=0)
scaled_xpr  = scale(xpressions)
print(scaled_xpr.mean(axis=0)[1], xpressions.mean(axis=0)[1])

-6.62938928373e-17 2145.36807554


In [134]:
rf = RandomForestClassifier(n_estimators=200)
svc = svm.SVC(kernel="linear", C=100)


# full_svc_smtr_a, full_svc_smtr_b = do_simmetry_class_cv(scaled_xpr,
#                                                      responses, svc, iters=10)

# intr_svc_smtr_a, intr_svc_smtr_b = do_simmetry_class_cv(scaled_xpr[:, intersection_gens],
#                                                     responses, svc, iters=10)

# full_rf_smtr_a, full_rf_smtr_b = do_simmetry_class_cv(xpressions,
#                                                      responses, rf, iters=10)

# intr_rf_smtr_a, intr_rf_smtr_b = do_simmetry_class_cv(xpressions[:, intersection_gens],
#                                                      responses, rf, iters=10)

# best31_rf_smtr_a, best31_rf_smtr_b = do_simmetry_class_cv(xpressions[:, best_31],

#                                                      responses, rf, iters=10)
# best31_svc_smtr_a, best31_svc_smtr_b = do_simmetry_class_cv(order_features3, responses, svc, iters=10)

Предообработка

In [9]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
clf = SVC(kernel="linear", probability=True)
xpressions_scaled = scale(xpressions)
# xpressions_scaled
# pro
# сv = 1responses

Классификация по генам:



In [93]:
np.random.seed(3267)
a,b, cv_results, descriptions  = do_simmetry_class_cv(xpressions_scaled,
                                                      responses, 
                                                      clf,
                                                      cv=cv, 
                                                      feature_maker=heuristics_1_2_features)
process_results(a, b, cv_results)

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9


,precision 0,recall 0,precision 1,recall 1,accuracy,auc
схема а,0.731599,0.720000,0.734468,0.698667,0.709333,0.773789
схема б,0.705847,0.717857,0.712986,0.700000,0.708929,0.766135


In [209]:
np.random.seed(3267)
a,b, cv_results, descriptions  = do_simmetry_class_cv(xpressions_scaled,
                                                      responses, 
                                                      knn,
                                                      cv=cv, 
                                                      feature_maker=heuristics_1_2_features)
process_results(a, b, cv_results)

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9


,precision 0,recall 0,precision 1,recall 1,accuracy,auc
схема а,0.445639,0.483667,0.445790,0.431667,0.457667,0.435772
схема б,0.450523,0.483929,0.454915,0.426786,0.455357,0.435571


In [307]:
# knn = KNeighborsClassifier(n_neighbors=8)
def get_h1_h2_gens(descr, gens):
    return [gens[int(g)] for g in descr.split("\n")[-1].split("\t")]

with open("evristiki_1_2.tsv", "w") as f:
    for descr in descriptions[0]:
        f.write("\t".join(get_h1_h2_gens(descr, gens)))
        f.write("\n")
    

In [110]:
gens_i = set([int(g) for g in descriptions[0][0].split("\n")[2].split("\t")])
for d in descriptions[0]:
    new_gens = set([int(g) for g in d.split("\n")[2].split("\t")])
    gens_i = gens_i.intersection(new_gens)
    print(len(new_gens))


print(gens_i, len(gens_i))


346
346
346
346
346
346
345
346
346
346
{2048, 10251, 12816, 19, 12820, 8216, 13354, 46, 6702, 12846, 13361, 9268, 53, 2103, 5176, 62, 7744, 11844, 3654, 8779, 12886, 1112, 3161, 9305, 5720, 3677, 8294, 615, 1640, 8810, 3697, 10868, 12918, 7287, 8836, 1671, 9865, 3211, 3728, 3218, 5783, 4764, 1181, 10412, 2733, 10925, 3254, 11459, 1747, 11992, 9955, 9444, 13540, 2278, 2800, 1265, 7409, 6388, 10997, 1273, 7932, 8450, 5893, 4360, 3338, 4875, 11531, 6927, 3343, 5907, 3863, 4382, 7977, 817, 13617, 11063, 8007, 841, 5962, 10058, 1871, 11600, 11090, 5970, 13144, 861, 7006, 1889, 4457, 7018, 7019, 5484, 12141, 7538, 8570, 3965, 9600, 9602, 11651, 6018, 12162, 5003, 11148, 909, 9109, 6037, 11163, 7073, 11170, 12197, 7088, 7608, 10687, 11714, 1475, 4554, 4560, 2000, 12761, 2018, 8680, 7656, 11243, 5611, 1007, 7670, 12793, 12286} 128


In [112]:
len(gens_i.intersection(set(interscetion_h1_h2)))

26

In [117]:
"ready"

'ready'

Порядковые фичи:

In [34]:
np.random.seed(3267)
a_t, b_t, cv_results_t, descriptions_t  = do_simmetry_class_cv(xpressions,
                                                      responses, 
                                                      clf,
                                                      cv=cv, 
                                                      iters=1,
                                                      feature_maker=heuristics_4_features)
process_results(a_t, b_t, cv_results_t)

iteration: 0
heuristic 4 started
heuristic 4 started
heuristic 4 started
heuristic 4 started
heuristic 4 started
heuristic 4 started
heuristic 4 started
heuristic 4 started
heuristic 4 started
heuristic 4 started


,precision 0,recall 0,precision 1,recall 1,accuracy,auc
схема а,0.693333,0.786667,0.822143,0.666667,0.726667,0.767778
схема б,0.698413,0.785714,0.755102,0.660714,0.723214,0.723852


In [33]:
# np.random.seed(3267)
a_t2, b_t2, cv_results_t2, descriptions_t2  = do_simmetry_class_cv(xpressions,
                                                      responses, 
                                                      clf,
                                                      cv=cv, 
                                                      iters=1,
                                                      feature_maker=heuristics_4_features)
process_results(a_t2, b_t2, cv_results_t2)

iteration: 0
heuristic 4 started


KeyboardInterrupt: 

In [289]:
def get_gens_from_descr(descr, gens):
    pairs = [p.replace("compare_expr(", "").replace(")", "").split(", ")
             for p in descr.split("\t")]
    gp = []
    for p1, p2 in pairs:
        gp.append((gens[int(p1)], gens[int(p2)]))
    return gp   

def pairs_to_str(pairs):
    pairs = ["{} {}".format(p1, p2) for p1, p2 in pairs]
    return "\t".join(pairs)
    

In [291]:
# descriptions_t2[0][0].split("\t")
# [p.replace("compare_expr(", "").replace(")", "").split(", ")
#  for p in descriptions_t2[0][0].split("\t")]
with open("pairs.tsv", "w") as f:
    for descr in descriptions_t2[0]:
        f.write(pairs_to_str(get_gens_from_descr(descr, gens)))
        f.write("\n")
        


In [14]:
responses.shape

(278,)

In [29]:
cv = 10
np.random.seed(3267)
a_tr, b_tr, cv_results_tr, descriptions_tr  = do_simmetry_class_cv(xpressions,
                                                      responses, 
                                                      clf,
                                                      cv=cv, 
                                                      iters=1,
                                                      feature_maker=heuristics_4_features)
process_results(a_tr, b_tr, cv_results_tr)

iteration: 0
heuristic 4 started


KeyboardInterrupt: 

In [267]:
descr_set = set(descriptions_t[0][0].split("\t"))

for description in descriptions_t2:# + descriptions_t:
    for d in description:        
        descr = set(d.split("\t"))
        print(len(descr))
        descr_set = descr_set.intersection(descr)
    
    
print("\n", len(descr_set))
# process_results(a_t2 + a_t, b_t2 + b_t, cv_results_t2 + cv_results_t)

70
70
70
70
70
70
70
70
70
70

 11


Классификация по пересечению генов из эвристик 1, 2:

In [154]:
inter_h1_h2_gens = ['PDE6C', 'CEP57', 'PDE9A', 'CALU', 'TTK', 'AATF', 'MIR1244-1',
       'MIR1244-2', 'MIR1244-3', 'TMEM109', 'QKI', 'PDIA6', 'PDK1',
       'SNAPC3', 'BACE2', 'NAA50', 'TTYH1', 'EPHB3', 'RPIA', 'SF3B3',
       'GORASP2', 'NUP153', 'NUP155', 'LRRC59', 'PDSS1', 'HLA-G', 'BANK1',
       'ANP32E', 'FOCAD', 'XPNPEP1', 'FOLH1B', 'TUBB4B', 'RAB23', 'EPRS',
       'SUMO3', 'LSM2', 'IQCG', 'NUSAP1', 'NASP', 'CSNK2B', 'ABI1', 'PERP',
       'GPM6B', 'PES1', 'DNAJA1', 'SYK', 'CASP8AP2', 'HN1', 'CHI3L2',
       'ACACA', 'RPL39L', 'TXNDC5', 'BCL11A', 'ISG20', 'SYNRG', 'TYMS',
       'RPP21', 'HNRNPH3', 'RAD1', 'ESPL1', 'CHRM3', 'ITGA6', 'RPS21',
       'ACP1', 'GPR89A', 'GPR89B', 'LCN2', 'CTPS1', 'ITGB4', 'SNRPC',
       'SNRPD1', 'NDC80', 'MAD2L1', 'GPSM2', 'PHACTR2', 'UBE2J1',
       'RALGAPB', 'TAF1D', 'NDRG2', 'TAF5', 'DONSON', 'ITPR3', 'RPS7',
       'TNFAIP8L2-SCNM1', 'RANBP6', 'BLOC1S5-TXNDC5', 'TAP1', 'RAP2A',
       'RRAGD', 'ACTR3', 'ACTR3B', 'HPS5', 'RRM2', 'FZD7', 'SKP2', 'DPY30',
       'CCHCR1', 'NDUFB6', 'CKS1B', 'CKS2', 'MAGOHB', 'DROSHA', 'RASGRP3',
       'KARS', 'DSC2', 'SOX10', 'UGP2', 'ZIC1', 'CYB5R2', 'CCNC', 'TNS2',
       'CCND3', 'NELFB', 'HSPA14', 'TBCC', 'FAIM2', 'BRIX1', 'BRPF1',
       'FAM107A', 'TOMM70', 'NES', 'DUSP14', 'DUSP2', 'CLIC4', 'RBM28',
       'GTF2E2', 'MAP4K4', 'S100A1', 'DUSP6', 'ZNF131', 'CCT4', 'NT5DC2',
       'CCT6A', 'MAPK1', 'BTG3', 'NFIB', 'UQCRH', 'UQCRHL', 'PRPF3',
       'GTPBP8', 'SAC3D1', 'NFYA', 'PRPF40A', 'PRPF4B', 'TCF7L2', 'CMC2',
       'KCNN4', 'CMPK1', 'PRPS2', 'USP18', 'PABPC3', 'MRPL9', 'LMO4',
       'SAP30', 'GATA6', 'GYG1', 'MRPS17', 'PADI2', 'MRPS18A', 'MRPS2',
       'USP39', 'ADSL', 'PRRC2A', 'SPR', 'GBP1', 'ART3', 'TEAD4',
       'PAK1IP1', 'PLCB4', 'MSH2', 'HACD1', 'IFI16', 'LOC100506248',
       'HADHA', 'MSH6', 'IFI44L', 'RFC2', 'LOC100506639', 'RFC4', 'COCH',
       'NMI', 'TRDMT1', 'FAM98A', 'C1orf112', 'SREK1', 'FANCG', 'PSIP1',
       'RFWD3', 'FANCL', 'MCM3', 'VCP', 'MCM6', 'TFDP2', 'CDC45', 'PSMB4',
       'TFF2', 'EFNA4', 'PLK4', 'VEGFA', 'CDCA8', 'SLC43A3', 'SCNM1',
       'EFS', 'SRSF3', 'VGLL1', 'GGH', 'SCPEP1', 'VGLL4', 'KIF11', 'MEA1',
       'TRIM38', 'TRIM39', 'TRIM39-RPP21', 'KIF14', 'KIF15', 'FBXO11',
       'DDX18', 'MTMR2', 'PSME4', 'PSMG1', 'SSR1', 'MED17', 'KIF20A',
       'HDAC9', 'MTO1', 'HDGF', 'PMPCA', 'SEC13', 'TRMT11', 'VPS72',
       'MELK', 'MEMO1', 'CDKN2A', 'C6orf62', 'ATP5G3', 'FDPS', 'DEK',
       'CDR2L', 'KLF11', 'IL12RB2', 'STAG1', 'FEN1', 'SEL1L3', 'DENND4B',
       'EIF5B', 'PTMA', 'KLF6', 'LOC389906', 'ELAVL1', 'KLHDC3', 'HIC2',
       'MEX3C', 'MYCN', 'ELF5', 'ZNF74', 'MFGE8', 'CEBPG', 'GMDS', 'CECR5',
       'STEAP1B', 'GMIP', 'LOC728026', 'RNF138', 'TSPAN6', 'TSPO', 'WDR77',
       'SERBP1', 'NRTN', 'SMC5', 'MYO10', 'STMN1', 'CENPM', 'DHTKD1',
       'ILF2', 'STOML2', 'LPIN1', 'PUS1', 'MICAL3', 'TTC13', 'PWP2',
       'CRLF1']
ids = np.arange(gens.shape[0])
interscetion_h1_h2 = sorted([ids[gen==gens][0] for gen in inter_h1_h2_gens if (gen == gens).sum() == 1])
# print(interscetion_h1_h2)

np.random.seed(3267)
a, b, cv_results = do_simmetry_class_cv(xpressions_scaled[:, interscetion_h1_h2], inv_responses, clf, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9


,precision 1,recall 1,precision 0,recall 0,accuracy,auc
схема а,0.725000,0.65,0.766667,0.85,0.75,0.5875
схема б,0.796667,0.65,0.730000,0.85,0.75,0.5875


In [143]:
len(inter_h1_h2_gens)

281

In [155]:
np.random.seed(3267)
knn = KNeighborsClassifier(n_neighbors=1)
a,b, cv_results  = do_simmetry_class_cv(xpressions_scaled[:, interscetion_h1_h2], inv_responses, knn, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9


,precision 1,recall 1,precision 0,recall 0,accuracy,auc
схема а,0.758333,0.575,0.708333,0.875,0.725,0.725
схема б,0.758333,0.575,0.701190,0.875,0.725,0.725


Случайно отобранные гены

In [156]:
np.random.seed(3267)
rnd_h1_h2 = np.random.choice(ids, size=len(interscetion_h1_h2), replace=False)
a, b, cv_results = do_simmetry_class_cv(xpressions_scaled[:, rnd_h1_h2], inv_responses, clf, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9


,precision 1,recall 1,precision 0,recall 0,accuracy,auc
схема а,0.783333,0.725,0.666667,0.675,0.7,0.2500
схема б,0.741667,0.725,0.708333,0.675,0.7,0.2875


In [157]:
np.random.seed(3267)
knn = KNeighborsClassifier(n_neighbors=1)
a,b, cv_results  = do_simmetry_class_cv(xpressions_scaled[:, rnd_h1_h2], inv_responses, knn, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9


,precision 1,recall 1,precision 0,recall 0,accuracy,auc
схема а,0.491667,0.475,0.458333,0.525,0.5,0.5
схема б,0.515000,0.475,0.488333,0.525,0.5,0.5


Классификация по 70 парам из эвристики 4

In [159]:
good_pairs_str = [('APBB3', 'CRLF1'), ('ARHGAP1', 'CRLF1'), ('ATP6V0A1', 'IL12RB2'), ('ATP6V0C', 'DUSP14'), ('C4A', 'NKTR'),
              ('C4B', 'NKTR'), ('C4B_2', 'NKTR'), ('CCND1', 'BCOR'), ('CCND1', 'MON1B'), ('CCND1', 'MTRF1'), ('CCND1', 'ZNF165'),
              ('CDK16', 'COCH'), ('CERS6', 'CRIPT'), ('CERS6', 'JMJD1C'), ('CERS6', 'MTO1'), ('CERS6', 'TLE1'),
              ('CERS6', 'TOPORS-AS1'), ('COL1A1', 'FOLH1B'), ('DHCR24', 'COCH'), ('DHCR24', 'FOCAD'), ('DNASE2', 'GPM6B'),
              ('EIF4B', 'ITGA6'), ('ERC1', 'CRLF1'), ('FHOD1', 'CRLF1'), ('FILIP1L', 'CRLF1'), ('FKSG49', 'MFGE8'),
              ('FOLR1', 'COCH'), ('GOLGA1', 'CRLF1'), ('GON4L', 'CRLF1'), ('GP1BA', 'CRLF1'), ('HOXB5', 'CRLF1'),
              ('HOXC4', 'FAIM2'), ('IGFBP4', 'DEPDC5'), ('IGFBP4', 'PIK3CG'), ('LYST', 'KCNAB1'), ('MIR6831', 'CRLF1'),
              ('MOSPD2', 'CRLF1'), ('MXRA8', 'GATA6'), ('MYL4', 'CRLF1'), ('MZT2B', 'ITGA6'), ('NAT1', 'PAQR3'),
              ('NDUFB5', 'MELK'), ('NECAB3', 'FBXO11'), ('NXPH3', 'CRLF1'), ('PEA15', 'NFIB'), ('PLXNA3', 'CRLF1'),
              ('PNPO', 'RWDD3'), ('POLDIP2', 'COCH'), ('PPIE', 'IL12RB2'), ('R3HDM4', 'ITGA6'), ('RALA', 'GPM6B'),
              ('RASA1', 'YES1'), ('RUFY1', 'ITGA6'), ('SEMA3C', 'BCOR'), ('SEMA3C', 'YIPF4'), ('SNORA67', 'ITGA6'),
              ('SPR', 'DNPH1'), ('STAMBP', 'MSH2'), ('SULT1A2', 'MRPS2'), ('THEMIS2', 'CRLF1'), ('TM6SF1', 'ELAVL2'),
              ('TMEM62', 'ETAA1'), ('TNK1', 'FBXO11'), ('TPBG', 'COCH'), ('TPBG', 'WSB1'), ('TRANK1', 'CRLF1'),
              ('TRPM2', 'CRLF1'), ('VPS35', 'MELK'), ('ZDHHC7', 'ITGA6'), ('ZNF510', 'CRLF1')]
gens_list = gens.tolist()
good_pairs = [(gens_list.index(gen1), gens_list.index(gen2)) for gen1, gen2 in good_pairs_str
              if (gen1 in gens_list) and (gen2 in gens_list)]

order_features1, pair_names = xpression_to_order_feature(xpressions, good_pairs, gens)
print(order_features1.shape)

np.random.seed(3267)
a, b, cv_results = do_simmetry_class_cv(order_features1, inv_responses, clf, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

(14, 67)
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9


,precision 1,recall 1,precision 0,recall 0,accuracy,auc
схема а,0.825000,0.875,0.916667,0.825,0.85,0.52500
схема б,0.828333,0.875,0.895000,0.825,0.85,0.56875


In [160]:
np.random.seed(3267)
knn = KNeighborsClassifier(n_neighbors=1)
a,b, cv_results  = do_simmetry_class_cv(order_features1, inv_responses, knn, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9


,precision 1,recall 1,precision 0,recall 0,accuracy,auc
схема а,0.725,0.625,0.750000,0.85,0.7375,0.7375
схема б,0.780,0.625,0.726667,0.85,0.7375,0.7375


In [149]:
def get_splits(responses, k=10):
    nonresp = np.arange(responses.shape[0])[responses==0]
    resp = np.arange(responses.shape[0])[responses==1]
    np.random.shuffle(nonresp)
    np.random.shuffle(resp)
    folds = []
    for i in range(9):
        folds.append(np.hstack((resp[i*22:i*22+22],nonresp[i*5:i*5+5])))
    folds.append(np.hstack((resp[9*22:],nonresp[9*5:])))
    cases = []
    for i in range(10):
        test = folds[i]
        train = np.hstack((fold for j, fold in enumerate(folds) if i != j))
        cases.append( (train, test) )                      
    return folds, cases
    

In [150]:
folds, cases = get_splits(responses)

In [151]:
intr_svc_smtr_a, intr_svc_smtr_b = do_simmetry_class_cv(order_features2,
                                                      responses, svc, iters=10)
order_features.shape, order_features2.shape, order_features3.shape

NameError: name 'order_features2' is not defined

In [ ]:
print(intr_svc_smtr_a.mean(axis=0), intr_svc_smtr_b.mean(axis=0))
print()
# print(best31_svc_smtr_a.mean(axis=0), best31_svc_smtr_b.mean(axis=0))
# prec_0, recall_0, prec_1, recall_1

In [ ]:
print(intr_svc_smtr_a.mean(axis=0), intr_svc_smtr_b.mean(axis=0))
print()
# print(best31_svc_smtr_a.mean(axis=0), best31_svc_smtr_b.mean(axis=0))
# prec_0, recall_0, prec_1, recall_1

In [309]:
import gensim

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
Problem occurred during compilation with the command line below:
g++ -shared -g -D NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -m64 -DMS_WIN64 -IC:\Users\vikazorin.OTDEL114\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\core\include -IC:\Users\vikazorin.OTDEL114\AppData\Local\Continuum\Anaconda3\include -o C:\Users\vikazorin.OTDEL114\AppData\Local\Theano\compiledir_Windows-7-6.1.7601-SP1-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64\lazylinker_ext\lazylinker_ext.pyd C:\Users\vikazorin.OTDEL114\AppData\Local\Theano\compiledir_Windows-7-6.1.7601-SP1-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.5.2-64\lazylinker_ext\mod.cpp -LC:\Users\vikazorin.OTDEL114\AppData\Local\Continuum\Anac

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xad in position 6: invalid start byte